# This notebook is for making phase plots of simulations

In [ ]:
##########################################
# Script for making phase plots
##########################################


# imports and parallelism
import yt
import numpy as np

yt.enable_parallelism()

# list of fields I'm interested in 
fields = ['density', 'temperature', 'entropy', 'metallicity', 'radial_velocity', 'cooling_time']

# make the timeseries
ts = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/h2sf/DD????/DD????")

# loop through all the snapshots in the sim
for ds in ts.piter():
    filename = ds.filename.split('/')[-1]
    
    
    # make a sphere for the phase plots. Should be about 2xVirialRadius. Radius may change depending on sim used
    myphasesphere = ds.sphere([0.5,0.5,0.5], (412,'kpc'))
    
    # get the time to add a timestamp
    time = str(np.round(ds.current_time.to('Gyr'), 1))

    # loop through all the fields you want
    for field in fields:
    
        phase = yt.PhasePlot(myphasesphere, ("index", "radius"), ("gas", field), 
                             ("gas", "mass"), weight_field=None)
        
        # set colorbar to stay constant
        # add timestamp manually
        if field == 'density':
            phase.set_ylim(ymin=1e-31, ymax=1e-24)
            phase.annotate_text(xpos=1e1, ypos=1.5e-31, text=time+' Gyr', color='black', fontsize=20)
        elif field == 'temperature':
            phase.set_ylim(ymin=1e3, ymax=1e8)
            phase.annotate_text(xpos=1e1, ypos=1.5e3, text=time+' Gyr', color='black', fontsize=20)
        elif field == 'entropy':
            phase.set_ylim(ymin=1e-1, ymax=1e5)
            phase.annotate_text(xpos=1e1, ypos=1.5e-1, text=time+' Gyr', color='black', fontsize=20)
        elif field == 'metallicity':
            phase.set_log(field, False)
            phase.set_ylim(ymin=0, ymax=7)
            phase.annotate_text(xpos=1e1, ypos=0.5, text=time+' Gyr', color='black', fontsize=20)
        elif field == 'radial_velocity':
            phase.set_unit(('gas', 'radial_velocity'), 'km/s')
            phase.set_log(field, False)
            phase.set_ylim(ymin=-100, ymax=100)
            phase.annotate_text(xpos=1e1, ypos=-90, text=time+' Gyr', color='black', fontsize=20)
        elif field == 'cooling_time':
            phase.set_unit('cooling_time', 'Myr')
            phase.set_ylim(ymin=1e1, ymax=1e8)
            phase.annotate_text(xpos=1e1, ypos=1.5e1, text=time+' Gyr', color='black', fontsize=20)
        
        # Set the units for mass
        phase.set_unit(("gas", 'mass'), 'Msun')
        phase.set_zlim(('gas', 'mass'), zmin=1e-1, zmax=1e8)
        
        # Set the units of radius to be in kpc
        phase.set_unit(("index", "radius"), "kpc")
        phase.set_log('radius', False)
        phase.set_xlim(xmin=0, xmax=420)
        
        phase.save('h2sf_'+str(field)+'_phaseplot_snapshot'+filename+'.png')
        
############################################        
        
        temp_dens = yt.PhasePlot(myphasesphere, ('gas','density'), ('gas', 'temperature'), 
                                 ('gas', 'mass'), weight_field=None)
        
        # Set the units for mass
        temp_dens.set_unit(("gas", 'mass'), 'Msun')
    
        temp_dens.set_xlim(xmin=1e-31, xmax=1e-24)
        temp_dens.set_ylim(ymin=1e3, ymax=1e8)
        temp_dens.set_zlim(('gas', 'mass'), zmin=1e-1, zmax=1e8)
        temp_dens.annotate_text(xpos=1e-23, ypos=1e7, text=time+' Gyr', color='black', fontsize=20)
        
        temp_dens.save('h2sf_TempVsDensity_phaseplot_snapshot'+filename+'.png')